In [18]:
# import packages
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import Bio
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
import statsmodels.api as sm
from scipy.optimize import curve_fit
import pylab
from sklearn.cluster import AffinityPropagation
from Levenshtein import distance



In [19]:
# import data from computer
path: str = './data/' #folder where files are stored
prokaryotes: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_unique_prot.csv"), dtype=str)   # imports file from the folder as species, based on name
prokaryotes_all: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_all.csv"), dtype=str) # imports 3rd file from the folder, based on name

prokaryotes_all['temperature'] = prokaryotes_all['temperature'].astype('float64')
prokaryotes_all['fold_change'] = prokaryotes_all['fold_change'].astype('float64')
prokaryotes_all['Gene Ontology IDs'] = prokaryotes_all['Gene Ontology IDs'].astype('str')

In [ ]:
GO_terms = []
current_term = ""

with open(os.path.join(path, "GO_annotation_list.obo"), 'r') as f:
    for line in f:
      line = line.strip()  
      if line.startswith('[Term]'):
        if current_term:
          GO_terms.append(current_term.strip())  
        current_term = ""
      else:
        current_term += line + "\n"  
    if current_term:
      GO_terms.append(current_term.strip())

GO_terms = np.asarray(GO_terms)   

In [20]:
def parse_obo(file_path):
    terms = {}
    current_term = {}
    isterm = False
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            isterm = isterm or line.startswith('id:')
            if isterm and ": " in line:
                key, value = line.split(': ', 1)
                if key == "id":
                    current_term = terms.setdefault(value, {})
                    current_term["id"] = value
                else:
                    current_term.setdefault(key, []).append(value)
    return terms

def make_hierarchy(terms):
    for term in list(terms.values()):
        term.setdefault("children", [])
        if "is_a" in term:
            for is_a in term["is_a"]:
                parent = is_a.split()[0]
                terms.setdefault(parent, { 'id': parent }).setdefault("children", []).append(term)
    return [term for term in terms.values() if "is_a" not in term]
    
def display_hierarchy(terms, indent=""):
    for term in terms:
        print(f"{indent}{term['id']}")
        display_hierarchy(term['children'], indent + "  ")
                       

In [21]:
hierarchy = make_hierarchy(parse_obo(os.path.join(path, "GO_annotation_list.obo")))

TypeError: 'dict' object is not callable